
# Camera price prediction, determining importance of the specs and webscraping
The aim of this project is simple - webscrape website https://www.dpreview.com to look for camera models, their prices, quick specs. Then the data read to csv file, do cleaning and a preparation of the data. And finally, to do some machine learning to find out which features are more important and be able to predict prices of other models of cameras according their quick specs.

In [ ]:
# Load libraries
import requests
from bs4 import BeautifulSoup

In [2]:
# Acces our desirable website
r = requests.get ("https://www.dpreview.com/products/cameras/all?page=1")
# Put it into the soup
soup = BeautifulSoup(r.content)

C:\Users\milen\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\milen\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


If you are stucked, just find the right people to ask questions. I did. And yeah, my problem has changed a little bit. Instead of browsing every sub-website of each model, I decided that specs provided at "my" main webpage (the one mentioned at the beginnig of this notebook) would be enough. I decided so from two reasons:

1) obviously : this a ready-made nice code is from stackoverflow (answer to my question there) and i just love it!

2) for my small ML project is enough and later I'll can solve the initial problem (and in my opinion, those features in the list are the most important)

In [3]:
import requests, typing
class Camera2(typing.NamedTuple):
  info:typing.List[str]
  quicklook:str
  price:str

from bs4 import BeautifulSoup as soup
d = soup(requests.get('https://www.dpreview.com/products/cameras/all?page=1').text, 'html.parser')
headers = [['div', 'name'], ['div', 'specs'], ['div', 'prices']]
vals = [[(lambda x:getattr(x, 'text', 'N/A') if b != 'name' else [getattr(x, 'text', 'N/A'), i.a['href']])(i.find(a, {'class':b})) for a, b in headers] for i in d.find_all('td', {'class':'product'})]
final_result2 = [Camera2(*i) for i in vals]

In [4]:
for i in range(50):
    print(final_result2[i],"\n")

Camera2(info=['Canon PowerShot SX70 HS', 'https://www.dpreview.com/products/canon/compacts/canon_sx70hs'], quicklook='20 megapixels | 3″ screen | 21 – 1365 mm (65×)', price='Check prices') 

Camera2(info=['Fujifilm X-T3', 'https://www.dpreview.com/products/fujifilm/slrs/fujifilm_xt3'], quicklook='26 megapixels | 3″ screen | APS-C sensor', price='$1,499.00 - $2,898.00') 

Camera2(info=['Canon EOS R', 'https://www.dpreview.com/products/canon/slrs/canon_eos_r'], quicklook='30 megapixels | 3.2″ screen | Full frame sensor', price='$2,299.00 - $6,398.00') 

Camera2(info=['Sony Cyber-shot DSC-HX99', 'https://www.dpreview.com/products/sony/compacts/sony_dschx99'], quicklook='18 megapixels | 3″ screen | 24 – 720 mm (30×)', price='Check prices') 

Camera2(info=['Sony Cyber-shot DSC-HX95', 'https://www.dpreview.com/products/sony/compacts/sony_dschx95'], quicklook='18 megapixels | 3″ screen | 24 – 720 mm (30×)', price='Check prices') 

Camera2(info=['Nikon D3500', 'https://www.dpreview.com/product

Next step will to definy functions that would collect data to good looking list (camera_data)

In [107]:
def cal_price(x):
    price = final_result2[x].price.split("$")
   # print("This is how final_result2 looks like aftert the split:", price)
    
    if len(final_result2[x].price.split("$")) == 1:
   #     print("final_result2 is the case of lenght of 1")
        if price == ['Check prices']:
            price = "N/A"

    elif len(final_result2[x].price.split("$")) == 2:
      #  print("final_result2 is the case of lenght of 2")
        if len(price[1]) == 6:
            price = float(price[1])
        elif len(price[1]) == 8:
            price1 = str(price[1][0]) + str(price[1][2:])
            price = float(price1)

    elif len(final_result2[x].price.split("$")) == 3:
  #      print("final_result2 is the case of lenght of 3")
        price1 = price[1][:-2]
  #      print("The second item is:", price1)
        if len(price1) == 7:
            price1 = str(price1[:2]) + str(price1[2:])

        elif len(price1) == 9:
            price1 = str(price1[0]) + str(price1[2:])
    #    print("price1 after corrections is:", price1)    

        price2 = price[2] 
   #     print("The third item is:", price2)
        if len(price2) == 6:
            price2 = str(price2[:2]) + str(price2[2:])

        elif len(price2) == 8:
            price2 = str(price2[0]) + str(price2[2:])
    #    print("price2 after corrections is:", price1)  

        price = (float(price1) + float(price2)) / 2 
  #      print("Price is:", price)
    
    return price

for i in range(50):
    print(cal_price(i))

N/A
2198.5
4348.5
N/A
N/A
655.425
2720.425
4120.424999999999
1046.9850000000001
N/A
399.0
499.0
898.0
1019.4250000000001
115.995
N/A
1232.995
749.0
447.99
211.69
3097.0
788.495
N/A
N/A
2296.95
3098.0
997.99
747.99
649.0
N/A
699.0
174.0
2724.18
3295.0
2995.98
498.435
N/A
3605.0
1099.0
421.945
1742.52
1174.115
465.495
624.0
649.0
564.0
5095.424999999999
2194.995
2224.495
748.5


In [21]:
def form_info(x):
    info = str(final_result2[x].info).split(',')
    info[0] = info[0].lstrip("['").rstrip("'")
    info[1] = info[1].lstrip(" '").rstrip("']")
    return info[0], info[1]

for i in range(50):
    print(form_info(i))

('Canon PowerShot SX70 HS', 'https://www.dpreview.com/products/canon/compacts/canon_sx70hs')
('Fujifilm X-T3', 'https://www.dpreview.com/products/fujifilm/slrs/fujifilm_xt3')
('Canon EOS R', 'https://www.dpreview.com/products/canon/slrs/canon_eos_r')
('Sony Cyber-shot DSC-HX99', 'https://www.dpreview.com/products/sony/compacts/sony_dschx99')
('Sony Cyber-shot DSC-HX95', 'https://www.dpreview.com/products/sony/compacts/sony_dschx95')
('Nikon D3500', 'https://www.dpreview.com/products/nikon/slrs/nikon_d3500')
('Nikon Z6', 'https://www.dpreview.com/products/nikon/slrs/nikon_z6')
('Nikon Z7', 'https://www.dpreview.com/products/nikon/slrs/nikon_z7')
('Panasonic Lumix DC-LX100 II', 'https://www.dpreview.com/products/panasonic/compacts/panasonic_dclx100ii')
('Leica M10-P', 'https://www.dpreview.com/products/leica/slrs/leica_m10_p')
('Canon PowerShot SX740 HS', 'https://www.dpreview.com/products/canon/compacts/canon_sx740hs')
('Fujifilm XF10', 'https://www.dpreview.com/products/fujifilm/compac

In [112]:
def form_quick(x):
    quick = str(final_result2[i].quicklook).split("|")
    if len(quick) == 3:
        quick[0] = quick[0].lstrip("['").rstrip("'")
        quick[1] = quick[1].lstrip("'").rstrip("'")
        quick[2] = quick[2].lstrip("'").rstrip("']")
        quick0, quick1, quick2 = quick[0], quick[1], quick[2]
    elif len(quick) == 2:
        quick[0] = quick[0].lstrip("['").rstrip("'")
        quick[1] = quick[1].lstrip("'").rstrip("'")
        quick0, quick1, quick2 = quick[0], quick[1], "N/A"
    else:
        quick0, quick1, quick2 = "N/A", "N/A","N/A"
      
        
    return quick0, quick1, quick2

for i in range(50):
    print(form_quick(i))
 #   if form_quick(i) == None:
   #     print(i)
#    try:
 #       print(len(form_quick(i)))
  #  except:
   #     print(0)
#quick = str(final_result2[0].quicklook).split("|")
#print(quick[0].lstrip("['").rstrip("'"))

('20 megapixels ', ' 3″ screen ', ' 21 – 1365 mm (65×)')
('26 megapixels ', ' 3″ screen ', ' APS-C sensor')
('30 megapixels ', ' 3.2″ screen ', ' Full frame sensor')
('18 megapixels ', ' 3″ screen ', ' 24 – 720 mm (30×)')
('18 megapixels ', ' 3″ screen ', ' 24 – 720 mm (30×)')
('24 megapixels ', ' 3″ screen ', ' APS-C sensor')
('25 megapixels ', ' 3.2″ screen ', ' Full frame sensor')
('46 megapixels ', ' 3.2″ screen ', ' Full frame sensor')
('17 megapixels ', ' 3″ screen ', ' 24 – 75 mm (3.1×)')
('24 megapixels ', ' 3″ screen ', ' Full frame sensor')
('21 megapixels ', ' 3″ screen ', ' 24 – 960 mm (40×)')
('24 megapixels ', ' 3″ screen', 'N/A')
('20 megapixels ', ' 3″ screen ', ' 24 – 70 mm (2.9×)')
('16 megapixels ', ' 3.2″ screen ', ' 24 – 3000 mm (125×)')
('N/A', 'N/A', 'N/A')
('20 megapixels ', ' 3″ screen ', ' 24 – 360 mm (15×)')
('20 megapixels ', ' 3″ screen ', ' 24 – 200 mm (8.3×)')
('24 megapixels ', ' 3″ screen ', ' APS-C sensor')
('20 megapixels ', ' 3″ screen ', ' 28 – 128 

In [117]:
import re
# camera_data[] is a list where we store out tidy data
camera_data = []
for i in range(len(final_result2)):
    part = []
    # make format for "info" section
    info = str(final_result2[i].info).split(',')
    info[0] = info[0].lstrip("['").rstrip("'")
    info[1] = info[1].lstrip(" '").rstrip("']")
    
    # make format for "price" section
    price = cal_price(i)
    
    # we let it be as it is (it's a mixture of number or strings)
    # and later made this "categorical" values as numerical to our models
    quick = form_quick(i)
    
    part = [info[0], info[1], quick[0], quick[1], quick[2], price]   
       
    # append all to camera_data[] {Do I want it as a list of lists, or not? or should I have it as dataframe directly?}   
    camera_data.append(part)
    
print(camera_data[14])

['Fujifilm instax mini 90 NEO CLASSIC', 'https://www.dpreview.com/products/fujifilm/compacts/fujifilm_instax_mini_90', 'N/A', 'N/A', 'N/A', 115.995]


In [118]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data=camera_data)
print(df)

                                            0  \
0                     Canon PowerShot SX70 HS   
1                               Fujifilm X-T3   
2                                 Canon EOS R   
3                    Sony Cyber-shot DSC-HX99   
4                    Sony Cyber-shot DSC-HX95   
5                                 Nikon D3500   
6                                    Nikon Z6   
7                                    Nikon Z7   
8                 Panasonic Lumix DC-LX100 II   
9                                 Leica M10-P   
10                   Canon PowerShot SX740 HS   
11                              Fujifilm XF10   
12             Sony Cyber-shot DSC-RX100 V(A)   
13                        Nikon Coolpix P1000   
14        Fujifilm instax mini 90 NEO CLASSIC   
15                                Leica C-Lux   
16               Sony Cyber-shot DSC-RX100 VI   
17                            Fujifilm X-T100   
18      Panasonic Lumix DC-TS7 (Lumix DC-FT7)   
19                  